In [1]:
import glob
import cv2
import numpy as np

In [10]:
def load_images():
    filepaths = glob.glob('*.png')
    return [cv2.imread(filepath, cv2.IMREAD_GRAYSCALE) for filepath in filepaths]

def find_principal_axis(image):
    _, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    moments = cv2.moments(thresh)
    centroid = (int(moments['m10']/moments['m00']), int(moments['m01']/moments['m00']))

    u20 = moments['mu20'] / moments['m00']
    u02 = moments['mu02'] / moments['m00']
    u11 = moments['mu11'] / moments['m00']

    angle = 0.5 * np.arctan2(2 * u11, (u20 - u02))
    return centroid, angle

def rotate_image(image, centroid, angle):
    height, width = image.shape
    rotation_matrix = cv2.getRotationMatrix2D(centroid, angle * 180 / np.pi, 1)
    return cv2.warpAffine(image, rotation_matrix, (width, height))

def conditional_reverse(image, centroid, angle):
    height, width = image.shape
    perpendicular_angle = angle + np.pi / 2
    line_end = (int(centroid[0] + 1000 * np.cos(perpendicular_angle)),
                int(centroid[1] + 1000 * np.sin(perpendicular_angle)))

    if line_end[1] > height / 2:
        return cv2.flip(image, 1)  # Horizontal flip
    return image

def process_images():
    images = load_images()
    for image in images:
        centroid, angle = find_principal_axis(image)
        image = rotate_image(image, centroid, angle)
        image = conditional_reverse(image, centroid, angle)
        cv2.imshow('Processed Image', image)
        cv2.waitKey(0)

    cv2.destroyAllWindows()

In [12]:
process_images()